Train a custom object detection model

##Preparation Set up

Installation of dependencies

In [ ]:
!pip install -q tflite-model-maker-nightly
!pip install -q tflite-support-nightly

     |████████████████████████████████| 617 kB 4.2 MB/s 
     |████████████████████████████████| 6.4 MB 50.7 MB/s 
     |████████████████████████████████| 234 kB 57.8 MB/s 
     |████████████████████████████████| 1.1 MB 58.1 MB/s 
     |████████████████████████████████| 1.2 MB 45.2 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 68.2 MB/s 
     |████████████████████████████████| 1.1 MB 54.2 MB/s 
     |████████████████████████████████| 120 kB 75.6 MB/s 
     |████████████████████████████████| 840 kB 61.6 MB/s 
     |████████████████████████████████| 3.4 MB 26.3 MB/s 
     |████████████████████████████████| 25.3 MB 3.7 MB/s 
     |████████████████████████████████| 1.1 MB 46.4 MB/s 
     |████████████████████████████████| 99 kB 7.8 MB/s 
     |████████████████████████████████| 352 kB 45.6 MB/s 
     |████████████████████████████████| 47.8 MB 61 kB/s 
     |████████████████████████████████| 462 kB 60.3 MB/s 
     |██████████████

Importing the required packages

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

##Prepering the dataset

Explaine dataset: size and description

In [ ]:
#Download the dataset here (path)
#!wget https://drive.google.com/file/d/1necrNwpHu0XV7HEA7Za9UpHUpRcJGg1u/view?usp=sharing
#!unzip -q android_figurine.zip

## Training the object detection model

**Step 1: Load the dataset**


*   Images train_data used to train the object detection model.
*   Images in val_data used to check the model generalization to new images



In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'RoadDamage/train_',
    'RoadDamage/train_',
    ['RoadDamage',]
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'RoadDamage/validate',
    'RoadDamage/validate',
    ['RoadDamage',]
)

##**Step 2: Select a model architecture**
EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models from EffientDet architecture. Each different on the perfomance (Size, Latency ms, Average Precision).

In [ ]:
#spec = model_spec.get('efficientdet_lite0')
#spec = model_spec.get('efficientdet_lite2')
spec = model_spec.get('efficientdet_lite3')

##Step 3: Train the TensorFlow model with training data


*   Set `epochs = 20`, to run 20 iterations the training set. Time to check the accuracy and stop validation loss `val_loss` stop decreasing to avoid overfitting
*   Set `batch_size = 4` to devide the training steps into 4
*   Set `train_whole_model = True` to fine-tune the whole model instead of just training the head layer to improve accuacy. The trade-off is: it may take longer to train.





In [ ]:
"""
with detection_graph.as_default():
       od_graph_def = tf.compat.v1.GraphDef()
       with tf.compat.v1.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
           serialized_graph = fid.read()
           od_graph_def.ParseFromString(serialized_graph)
           tf.compat.v1.import_graph_def(od_graph_def, name='') """

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
36/36 [==============================] - 552s 13s/step - det_loss: 1.6977 - cls_loss: 1.0808 - box_loss: 0.0123 - reg_l2_loss: 0.0926 - loss: 1.7902 - learning_rate: 0.0065 - gradient_norm: 1.1343 - val_det_loss: 1.6680 - val_cls_loss: 1.0229 - val_box_loss: 0.0129 - val_reg_l2_loss: 0.0925 - val_loss: 1.7605
Epoch 2/20
36/36 [==============================] - 495s 14s/step - det_loss: 1.3705 - cls_loss: 0.7932 - box_loss: 0.0115 - reg_l2_loss: 0.0926 - loss: 1.4630 - learning_rate: 0.0049 - gradient_norm: 3.1052 - val_det_loss: 1.3373 - val_cls_loss: 0.6927 - val_box_loss: 0.0129 - val_reg_l2_loss: 0.0926 - val_loss: 1.4298
Epoch 3/20
36/36 [==============================] - 487s 14s/step - det_loss: 1.1612 - cls_loss: 0.6142 - box_loss: 0.0109 - reg_l2_loss: 0.0926 - loss: 1.2537 - learning_rate: 0.0048 - gradient_norm: 3.4311 - val_det_loss: 1.3134 - val_cls_loss: 0.6812 - val_box_loss: 0.0126 - val_reg_l2_loss: 0.0926 - val_loss: 1.4060
Epoch 4/20
36/36 [================

##Step 4: Evaluate the model with the validation data.
The evaluation metrics are same as COCO

In [ ]:
model.evaluate(val_data)

##Step 5: Export as a TensorFlow Lite model

In [ ]:
model.export(export_dir='.', tflite_filename = 'atom2.tflite')

##Step 6: Evaluate the TensorFlow Lite model

In [ ]:
model.evaluate_tflite('atom.tflite', val_data)

15/15 [==============================] - 47s 3s/step



{'AP': 0.09899021,
 'AP50': 0.31143466,
 'AP75': 0.023227543,
 'AP_/RoadDamage': 0.09899021,
 'APl': 0.10100878,
 'APm': 0.0,
 'APs': -1.0,
 'ARl': 0.22417583,
 'ARm': 0.0,
 'ARmax1': 0.029032258,
 'ARmax10': 0.18387097,
 'ARmax100': 0.21935484,
 'ARs': -1.0}

##Step 7: Download the TFLite model

In [ ]:
from google.colab import files
files.download('atom.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>